In [1]:
import requests
import pandas as pd
import datetime

api_key = "2ba3adde85df148b0ed86632635963e6" 
url = f"https://api.openweathermap.org/data/2.5/weather?"

def get_weather_data(city, country):
    params = {
        "q": f"{city},{country}",
        "appid": api_key,
        "units": "metric"
    }
    response = requests.get(url, params=params)
    response.raise_for_status() 
    return response.json()

def transform_weather_data(data):
    # Extract relevant data
    transformed_data = []
    weather_dict = {
        "timestamp": datetime.datetime.fromtimestamp(data["dt"]),
        "city": data["name"],
        "country": data["sys"]["country"],
        "temperature": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "wind_speed": data["wind"]["speed"],
        "weather_description": data["weather"][0]["description"]
    }
    transformed_data.append(weather_dict)
    dataframe = pd.DataFrame(transformed_data)
    return dataframe

# List of city-country pairs
city_country_pairs = [
    ("London", "UK"),
    ("Paris", "FR"),
    ("New York", "US"),
    ("Tokyo", "JP")
]

# Create an empty list to store dataframes
all_dataframes = []

# Iterate through the city-country pairs
for city, country in city_country_pairs:
    try:
        data = get_weather_data(city, country)
        dataframe = transform_weather_data(data)
        all_dataframes.append(dataframe)
    except requests.exceptions.RequestException as e:
        print(f"API request failed for {city}, {country}: {e}")

# Concatenate all dataframes into a single dataframe
combined_dataframe = pd.concat(all_dataframes, ignore_index=True)

# Print the combined dataframe
print(combined_dataframe)

            timestamp      city country  temperature  humidity  wind_speed  \
0 2024-03-18 16:49:59    London      GB        13.11        83        3.60   
1 2024-03-18 16:50:07     Paris      FR        15.21        68        3.09   
2 2024-03-18 16:52:33  New York      US         5.87        50        5.14   
3 2024-03-18 16:49:24     Tokyo      JP         6.80        26       11.32   

  weather_description  
0     overcast clouds  
1     overcast clouds  
2       broken clouds  
3           clear sky  


In [2]:
! pip install psycopg2


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import psycopg2

try:

    conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="Neha@00281",
    port= '5432'
    )

    cursor = conn.cursor()

    # #Drop the table
    # dropTableStmt   = "DROP TABLE %s;"%"weather_data2";
    # # Execute the drop table command
    # cursor.execute(dropTableStmt);


    table_creation = """
    CREATE TABLE IF NOT EXISTS weather_data (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP NOT NULL,
    city VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    temperature FLOAT NOT NULL,
    humidity INTEGER NOT NULL,
    wind_speed FLOAT NOT NULL,
    weather_description VARCHAR(200) NOT NULL
    );
    """

    cursor.execute(table_creation)
    conn.commit()
    
    # Insert data from the DataFrame into the table
    for _, row in combined_dataframe.iterrows():
        query = """
            INSERT INTO weather_data (
                timestamp, city, country, temperature, humidity, wind_speed, weather_description
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s
            );
        """
        values = (
            row["timestamp"],
            row["city"],
            row["country"],
            row["temperature"],
            row["humidity"],
            row["wind_speed"],
            row["weather_description"]
        )
        cursor.execute(query, values)
        conn.commit()


except Exception as error:
    print(error)

finally:
    if cursor is None:
        cursor.close()
    if conn is not None:
        conn.close()


